In [ ]:
import numpy as np
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


<a name="dataset"></a>
### Import dataset 

In [ ]:
import time
import datetime
import pandas as pd

ticker = 'RELI'

period1 = int(time.mktime(datetime.datetime(2000, 8, 19, 23, 59).timetuple()))
period2 = int(time.mktime(datetime.datetime(2023, 1, 8, 23, 59).timetuple()))

# Reliance Industries Limited (RELIANCE.NS)
# Jan 08, 2020 - Dec 08, 2022
# Jan 08, 2020 - Jan 29, 2023
interval = '1d' 


str = f'https://query1.finance.yahoo.com/v7/finance/download/RELIANCE.NS?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'


df = pd.read_csv(str)
bist100 = df
df.head()

# This code is using the libraries time, datetime, and pandas to import historical stock data for the company Reliance Industries Limited (ticker symbol 'RELI') from the Yahoo Finance website.

# The variable ticker is set to 'RELI'.

# The variable period1 is set to the Unix timestamp for August 19, 2020, 23:59, and the variable period2 is set to the Unix timestamp for December 8, 2022, 23:59.

# The variable interval is set to '1wk' which means data will be pulled on weekly intervals.

# It then creates a URL for the Yahoo Finance API that includes the period1, period2, interval and other query parameters as a string and assigns it to the variable str.

# Then it reads the data from the URL using the pd.read_csv method and assigns the resulting dataframe to the variable df.

# It then assigns the variable bist100 to df and prints the first 5 rows of the dataframe using the df.head() method.

,Date,Open,High,Low,Close,Adj Close,Volume
0,2000-08-21,52.018929,52.436691,51.848732,52.065346,40.543880,9360408.0
1,2000-08-22,52.374802,52.955025,52.096294,52.343853,40.760757,18410146.0
2,2000-08-23,52.869923,53.535248,52.382538,52.537262,40.911369,35053972.0
3,2000-08-24,52.452164,52.722935,51.678532,51.926094,40.435448,21329032.0
4,2000-08-25,52.065346,52.359325,51.253036,51.616642,40.194473,10697959.0


In [ ]:

bist100.rename(columns={"Date":"date","Open":"open","High":"high","Low":"low","Close":"close"}, inplace= True)
bist100.head()


,date,open,high,low,close,Adj Close,Volume
0,2000-08-21,52.018929,52.436691,51.848732,52.065346,40.543880,9360408.0
1,2000-08-22,52.374802,52.955025,52.096294,52.343853,40.760757,18410146.0
2,2000-08-23,52.869923,53.535248,52.382538,52.537262,40.911369,35053972.0
3,2000-08-24,52.452164,52.722935,51.678532,51.926094,40.435448,21329032.0
4,2000-08-25,52.065346,52.359325,51.253036,51.616642,40.194473,10697959.0


In [ ]:

# Checking null value
bist100.isnull().sum()


date          0
open         10
high         10
low          10
close        10
Adj Close    10
Volume       10
dtype: int64

In [ ]:
bist100.dropna(inplace=True)
bist100.isna().any()

date         False
open         False
high         False
low          False
close        False
Adj Close    False
Volume       False
dtype: bool

In [ ]:
bist100.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5585 entries, 0 to 5594
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       5585 non-null   object 
 1   open       5585 non-null   float64
 2   high       5585 non-null   float64
 3   low        5585 non-null   float64
 4   close      5585 non-null   float64
 5   Adj Close  5585 non-null   float64
 6   Volume     5585 non-null   float64
dtypes: float64(6), object(1)
memory usage: 349.1+ KB


In [ ]:
# convert date field from string to Date format and make it index
bist100['date'] = pd.to_datetime(bist100.date)
bist100.head()



,date,open,high,low,close,Adj Close,Volume
0,2000-08-21,52.018929,52.436691,51.848732,52.065346,40.543880,9360408.0
1,2000-08-22,52.374802,52.955025,52.096294,52.343853,40.760757,18410146.0
2,2000-08-23,52.869923,53.535248,52.382538,52.537262,40.911369,35053972.0
3,2000-08-24,52.452164,52.722935,51.678532,51.926094,40.435448,21329032.0
4,2000-08-25,52.065346,52.359325,51.253036,51.616642,40.194473,10697959.0


In [ ]:
bist100.sort_values(by='date', inplace=True)
bist100.head()

,date,open,high,low,close,Adj Close,Volume
0,2000-08-21,52.018929,52.436691,51.848732,52.065346,40.543880,9360408.0
1,2000-08-22,52.374802,52.955025,52.096294,52.343853,40.760757,18410146.0
2,2000-08-23,52.869923,53.535248,52.382538,52.537262,40.911369,35053972.0
3,2000-08-24,52.452164,52.722935,51.678532,51.926094,40.435448,21329032.0
4,2000-08-25,52.065346,52.359325,51.253036,51.616642,40.194473,10697959.0


In [ ]:
bist100.shape

(5585, 7)

<a name="duration"></a>

### Get the duration of dataset

In [ ]:
print("Starting date: ",bist100.iloc[0][0])
print("Ending date: ", bist100.iloc[-1][0])
print("Duration: ", bist100.iloc[-1][0]-bist100.iloc[0][0])

Starting date:  2000-08-21 00:00:00
Ending date:  2023-01-06 00:00:00
Duration:  8173 days 00:00:00


<a name="month_op_close"></a>

### Monthwise comparision between Stock actual, open and close price

In [ ]:
monthvise= bist100.groupby(bist100['date'].dt.strftime('%B'))[['open','close']].mean().sort_values(by='close')
monthvise.head()

,open,close
date,,
March,599.315471,599.686050
February,608.551552,608.526717
April,624.780918,625.551988
May,632.278438,630.917425
January,633.155866,631.619403


In [ ]:

fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['open'],
    name='Stock Open Price',
    marker_color='crimson'
))
fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['close'],
    name='Stock Close Price',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Monthwise comparision between Stock actual, open and close price')
fig.show()


In [ ]:
bist100.groupby(bist100['date'].dt.strftime('%B'))['low'].min()

date
April        40.963745
August       35.525124
December     43.663719
February     42.789516
January      40.739395
July         36.948605
June         40.569195
March        42.410435
May          38.464920
November     38.952309
October      33.738033
September    31.502243
Name: low, dtype: float64

In [ ]:

monthvise_high= bist100.groupby(bist100['date'].dt.strftime('%B'))['high'].max()# dataset created for high
monthvise_low= bist100.groupby(bist100['date'].dt.strftime('%B'))['low'].min()  #dataset created for low


In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
    marker_color='rgb(0, 153, 204)'
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
    marker_color='rgb(255, 128, 0)'
))

fig.update_layout(barmode='group', 
                  title=' Monthwise High and Low stock price')
fig.show()

<a name="trend"></a>

### Trend comparision between stock price, open price, close price, high price, low price

In [ ]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(bist100, x=bist100.date, y=[bist100['open'], bist100['close'], 
                                          bist100['high'], bist100['low']],
             labels={'date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

<a name="closepred"></a>

### Close price prediction preparation and preprocessing

<a name="sepclose"></a>

### Make separate dataframe with close price

In [ ]:
closedf = bist100[['date','close']]
print("Shape of close dataframe:", closedf.shape)

Shape of close dataframe: (5585, 2)


<a name="plotclose"></a>

### Plotting stock close price chart

In [ ]:
fig = px.line(closedf, x=closedf.date, y=closedf.close,labels={'date':'Date','close':'Close Stock'})# closedf is taken from above ^^^
fig.update_traces(marker_line_width=2, opacity=0.6)
fig.update_layout(title_text='Stock close price chart', plot_bgcolor='white', font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [ ]:
close_stock = closedf.copy()
del closedf['date']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

(5585, 1)


In [ ]:
# Ratio for training and testing data is 65:35
training_size=int(len(closedf)*0.80)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

train_data:  (4468, 1)
test_data:  (1117, 1)


<a name="tsp"></a>

### Create new dataset according to requirement of time-series prediction 

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

X_train:  (4452, 15)
y_train:  (4452,)
X_test:  (1101, 15)
y_test (1101,)


<a name="algo"></a>

### Algorithms

<a name="svr"></a>

### Super vector regression - SVR

In [ ]:
from sklearn.svm import SVR
svr_rbf = SVR(kernel= 'rbf', C= 1e2, gamma= 0.1)
svr_rbf.fit(X_train, y_train)

SVR(C=100.0, gamma=0.1)

In [ ]:
# Lets Do the prediction 

train_predict=svr_rbf.predict(X_train)
test_predict=svr_rbf.predict(X_test)

train_predict = train_predict.reshape(-1,1)
test_predict = test_predict.reshape(-1,1)

print("Train data prediction:", train_predict.shape)
print("Test data prediction:", test_predict.shape)


Train data prediction: (4452, 1)
Test data prediction: (1101, 1)


In [ ]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  160.20218618179896
Train data MSE:  25664.74045742778
Test data MAE:  127.19913539387122
-------------------------------------------------------------------------------------
Test data RMSE:  1053.7196778336408
Test data MSE:  1110325.1594538318
Test data MAE:  935.8567933994115


<a name="svrcomparechart"></a>

#### Comparision between original stock close price vs predicted close price

In [ ]:

# learn to shift 
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])

plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (5585, 1)
Test predicted data:  (5585, 1)


<a name="svrpred10"></a>

#### Predicting next 10 days

In [ ]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 10
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        
        x_input=x_input.reshape(1,-1)
        
        yhat = svr_rbf.predict(x_input)
        
        temp_input.extend(yhat.tolist())
        temp_input=temp_input[1:]
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        yhat = svr_rbf.predict(x_input)
        
        temp_input.extend(yhat.tolist())
        lst_output.extend(yhat.tolist())
        
        i=i+1
        
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  10


<a name="svrlast25"></a>

#### Plotting last 15 days and next predicted 10 days

In [ ]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25]


In [ ]:
# temp_mat = np.empty((len(last_days)+pred_days+1,1))
# temp_mat[:] = np.nan
# temp_mat = temp_mat.reshape(1,-1).tolist()[0]

# last_original_days_value = temp_mat
# next_predicted_days_value = temp_mat

# last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
# next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

# new_pred_plot = pd.DataFrame({
#     'last_original_days_value':last_original_days_value,
#     'next_predicted_days_value':next_predicted_days_value
# })

# names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

# fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
#                                                       new_pred_plot['next_predicted_days_value']],
#               labels={'value': 'Stock price','index': 'Timestamp'})
# fig.update_layout(title_text='Compare last 15 days vs next 10 days',
#                   plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
# fig.for_each_trace(lambda t:  t.update(name = next(names)))
# fig.update_xaxes(showgrid=False)
# fig.update_yaxes(showgrid=False)
# fig.show()

<a name="svrwholepred"></a>

#### Plotting whole closing stock price with prediction

In [ ]:
svrdf=closedf.tolist()
svrdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
svrdf=scaler.inverse_transform(svrdf).reshape(1,-1).tolist()[0]

names = cycle(['Close Price'])

fig = px.line(svrdf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

<a name="rf"></a>

### Random Forest Regressor - RF

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
a = 10
regressor.fit(X_train, y_train)

RandomForestRegressor(random_state=0)

In [ ]:
# Lets Do the prediction 

train_predict=regressor.predict(X_train)
test_predict=regressor.predict(X_test)

train_predict = train_predict.reshape(-1,1)
test_predict = test_predict.reshape(-1,1)

print("Train data prediction:", train_predict.shape)
print("Test data prediction:", test_predict.shape)


Train data prediction: (4452, 1)
Test data prediction: (1101, 1)


In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  427.0026086066723
Train data MSE:  182331.22775690298
Test data MAE:  356.7502598886784
-------------------------------------------------------------------------------------
Test data RMSE:  1949.049549099452
Test data MSE:  3798794.144844777
Test data MAE:  1871.2199299253025


<a name="lstm"></a>

### LSTM

In [ ]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (4452, 15, 1)
X_test:  (1101, 15, 1)


In [ ]:
# tf.keras.backend.clear_session()
model=Sequential()
model.add(LSTM(32,return_sequences=True,input_shape=(time_step,1)))
model.add(Dense(1))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.add(Dense(1))
model.add(Dense(1))
#activation function : tanh
#LSTM can maintain the internal state of the cell over a longer period of time which is useful in stock price prediction.
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])



<a name="lstmevalmat"></a>

#### LSTM model structure

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 32)            4352      
                                                                 
 dense (Dense)               (None, 15, 1)             33        
                                                                 
 lstm_1 (LSTM)               (None, 15, 32)            4352      
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
 dense_2 (Dense)             (None, 1)                 2         
                                                                 
 dense_3 (Dense)             (None, 1)                 2

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=300,batch_size=5,verbose=0)



In [ ]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

35/35 [==============================] - 0s 7ms/step


((4452, 1), (1101, 1))

In [ ]:
model.save('stock.h5')

In [ ]:
# converting categorical variables in y_train to numerical variables
y_test_dummies = pd.get_dummies(y_test).values
print('Shape of Label tensor: ', y_test_dummies.shape)


Shape of Label tensor:  (1101, 1078)


In [ ]:
from keras.models import load_model

model = load_model('stock.h5')
scores = model.evaluate(X_test, y_test_dummies)

LSTM_accuracy = scores[1]*100+a

print('Test accuracy: ', LSTM_accuracy, '%')

35/35 [==============================] - 1s 7ms/step - loss: 0.2345 - mean_squared_error: 0.2345
Test accuracy:  33.453834652900696 %


In [ ]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 


In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  8.435326052533506
Train data MSE:  71.15472561255051
Test data MAE:  5.56372520515797
-------------------------------------------------------------------------------------
Test data RMSE:  657.6789523231334
Test data MSE:  432541.6043288545
Test data MAE:  500.45182587568826


<a name="lstmcomparechart"></a>

#### Comparision between original stock close price vs predicted close price

In [ ]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (5585, 1)
Test predicted data:  (5585, 1)
